# Model Building for MNIST

In [2]:
from theano.sandbox import cuda
cuda.use('gpu1')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [3]:
%matplotlib inline
from importlib import reload
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


## Setup

In [4]:
batch_size = 64
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

15302656/15296311 [==============================] - 2s     


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
# Because MNIST is grey-scale images, it does not have the color column,
# Let's add one empty dim  to the X data
X_test = np.expand_dims(X_test, 1)
X_train = np.expand_dims(X_train, 1)
X_train.shape

(60000, 1, 28, 28)

In [6]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [7]:
y_train = onehot(y_train)
y_test = onehot(y_test)
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

Now, let's normalize the inputs

In [8]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [9]:
def norm_input(x): return (x-mean_px)/std_px

## Linear model

Why not we just fine-tune the imagenet model?

Because imageNet is 214 x 214 and is full-color. Here we have 28 x 28 and greyscale.

So we need to start from scratch.

In [15]:
def get_lin_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

lm = get_lin_model()

/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [16]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [17]:
lm.fit_generator(batches, batches.N, nb_epoch=1, 
                validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 14s - loss: 0.4259 - acc: 0.8735 - val_loss: 0.3073 - val_acc: 0.9142


It's always recommended to start with epoch 1 and a low learning rate. Defaut is 0.0001

In [18]:
lm.optimizer.lr = 0.1
lm.fit_generator(batches, batches.N, nb_epoch=3,
                validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/3
60000/60000 [==============================] - 14s - loss: 0.2987 - acc: 0.9149 - val_loss: 0.2854 - val_acc: 0.9181
Epoch 2/3
60000/60000 [==============================] - 14s - loss: 0.2842 - acc: 0.9201 - val_loss: 0.2820 - val_acc: 0.9192
Epoch 3/3
60000/60000 [==============================] - 13s - loss: 0.2769 - acc: 0.9224 - val_loss: 0.2733 - val_acc: 0.9223


## Single Dense Layer

In [19]:
def get_fc_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(512, activation='softmax'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

fc = get_fc_model()

/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


As before, let's start with 1 epoch and a default low learning rate.

In [21]:
fc.fit_generator(batches, batches.N, nb_epoch=1, 
                validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 14s - loss: 1.5790 - acc: 0.8199 - val_loss: 1.0741 - val_acc: 0.8387


In [22]:
fc.optimizer.lr=0.01
fc.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 14s - loss: 0.8244 - acc: 0.8479 - val_loss: 0.6389 - val_acc: 0.8518
Epoch 2/4
60000/60000 [==============================] - 14s - loss: 0.5564 - acc: 0.8556 - val_loss: 0.4987 - val_acc: 0.8488
Epoch 3/4
60000/60000 [==============================] - 15s - loss: 0.4568 - acc: 0.8601 - val_loss: 0.4365 - val_acc: 0.8499
Epoch 4/4
60000/60000 [==============================] - 14s - loss: 0.4099 - acc: 0.8675 - val_loss: 0.4190 - val_acc: 0.8573


## Basic 'VGG-style' CNN

In [ ]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28, 28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [24]:
model = get_model()
model.fit_generator(batches, batches.N, nb_epoch=1,
                   validation_data=test_batches, nb_val_samples=test_batches.N)

/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_5 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Epoch 1/1
60000/60000 [==============================] - 22s - loss: 0.1063 - acc: 0.9672 - val_loss: 0.0312 - val_acc: 0.9899


In [25]:
model.optimizer.lr=0.1
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 22s - loss: 0.0361 - acc: 0.9888 - val_loss: 0.0304 - val_acc: 0.9904


In [26]:
model.optimizer.lr=0.01
model.fit_generator(batches, batches.N, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/8
60000/60000 [==============================] - 22s - loss: 0.0232 - acc: 0.9928 - val_loss: 0.0298 - val_acc: 0.9906
Epoch 2/8
60000/60000 [==============================] - 22s - loss: 0.0189 - acc: 0.9938 - val_loss: 0.0332 - val_acc: 0.9901
Epoch 3/8
60000/60000 [==============================] - 22s - loss: 0.0146 - acc: 0.9955 - val_loss: 0.0287 - val_acc: 0.9915
Epoch 4/8
60000/60000 [==============================] - 22s - loss: 0.0137 - acc: 0.9953 - val_loss: 0.0196 - val_acc: 0.9934
Epoch 5/8
60000/60000 [==============================] - 22s - loss: 0.0110 - acc: 0.9963 - val_loss: 0.0349 - val_acc: 0.9917
Epoch 6/8
60000/60000 [==============================] - 22s - loss: 0.0103 - acc: 0.9966 - val_loss: 0.0283 - val_acc: 0.9930
Epoch 7/8
60000/60000 [==============================] - 22s - loss: 0.0086 - acc: 0.9970 - val_loss: 0.0314 - val_acc: 0.9919
Epoch 8/8
60000/60000 [==============================] - 22s - loss: 0.0065 - acc: 0.9982 - val_loss: 0.0287 - 